# Data preparation: dataset `AR_dummy`

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

## Load and prepare data

**This dataframe includes the following data:**

**Nordecon**
 - Consolidated total revenue
 - Consolidated revenue in Estonia
 - Order book

**Gross Domestic Product (GDP)**
 - GDP at constant prices:
    * Private consumption
    * Government sector consumption
    * Gross fixed capital formation
    * Export of goods and services
    * Import of goods and services

**Labor market**
 - Unemployment rate
 - Employment rate
 - Average gross monthly wage

**Inflation**
 - Consumer Price Index (CPI)
 - Producer Price Index (PPI)

**Balance of payments**
 - Current account balance
 - Current account balance to GDP
 - Foreign direct investments

**State budget**
 - State budget surplus/deficit
 - Government sector budget balance
 - Government sector investments

**Confidence indicators**
 - Industrial enterprises' confidence indicator
 - Retail trade enterprises' confidence indicator
 - Services enterprises' confidence indicator
 - Consumer confidence indicator
 - Economic confidence index
 - Business sentiment indicator: demand in industry as limiting factor
 - Business sentiment indicator: demand in services as limiting factor

**Volume indices**
 - Retail trade volume index
 - Industrial production volume index
 - New car registrations
 - Sale of apartments

**Building permits**
 - Residential building permits
 - Non-residential building permits

**Productivity**
 - Labor productivity per employed person
 - Labor productivity per worked hour

**Corporate financing**
 - Total corporate profit
 - Total corporate investments into buildings and structures  
 - Loans granted to non-financial enterprises
 - M1 (money supply indicator)

In [ ]:
arima_df = pd.read_csv('trainvalidtest_dummy.csv')
arima_df = arima_df.iloc[:, :3]
# Combine "Year" and "Quarter" columns into a single datetime index
arima_df['End_of_Quarter'] = pd.PeriodIndex(arima_df['Year'].astype(str) + 'Q' + arima_df['Quarter'].astype(str), freq='Q').to_timestamp()
# Adjust to represent the end date of each quarter
arima_df['End_of_Quarter'] = arima_df['End_of_Quarter'] + pd.offsets.MonthEnd(3)
arima_df.drop(columns=['Year', 'Quarter'], inplace=True)
# Set the datetime index
arima_df.set_index('End_of_Quarter', inplace=True)
arima_df

,NCN_sales_est
End_of_Quarter,
2008-03-31,40446.0
2008-06-30,54576.0
2008-09-30,54635.0
2008-12-31,48859.0
2009-03-31,31236.0
...,...
2022-12-31,79376.0
2023-03-31,46742.0
2023-06-30,76021.0


In [ ]:
# Number of quarters to keep for validation
validation_quarters = 4

splits_val = {'train': [], 'val': []}

for i in range(len(arima_df)-19, len(arima_df) - validation_quarters, 1):
    train_data = arima_df.iloc[:i + validation_quarters]
    valid_data = arima_df.iloc[i + validation_quarters:i + validation_quarters + validation_quarters]
    print(f'TRAIN: {train_data.index}, VALID: {valid_data.index}')
    splits_val['train'].append(train_data)
    splits_val['val'].append(valid_data)

    if i + 11 >= len(arima_df) - 1:
        break

TRAIN: DatetimeIndex(['2008-03-31', '2008-06-30', '2008-09-30', '2008-12-31',
               '2009-03-31', '2009-06-30', '2009-09-30', '2009-12-31',
               '2010-03-31', '2010-06-30', '2010-09-30', '2010-12-31',
               '2011-03-31', '2011-06-30', '2011-09-30', '2011-12-31',
               '2012-03-31', '2012-06-30', '2012-09-30', '2012-12-31',
               '2013-03-31', '2013-06-30', '2013-09-30', '2013-12-31',
               '2014-03-31', '2014-06-30', '2014-09-30', '2014-12-31',
               '2015-03-31', '2015-06-30', '2015-09-30', '2015-12-31',
               '2016-03-31', '2016-06-30', '2016-09-30', '2016-12-31',
               '2017-03-31', '2017-06-30', '2017-09-30', '2017-12-31',
               '2018-03-31', '2018-06-30', '2018-09-30', '2018-12-31',
               '2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31',
               '2020-03-31'],
              dtype='datetime64[ns]', name='End_of_Quarter', freq=None), VALID: DatetimeIndex(['2020-06-30', '2

In [ ]:
# Number of quarters to keep for validation
validation_quarters = 4

splits_test = {'train': [], 'test': []}

for i in range(len(arima_df)-15, len(arima_df) - validation_quarters, 1):
    train_data = arima_df.iloc[:i + validation_quarters]
    valid_data = arima_df.iloc[i + validation_quarters:i + validation_quarters + validation_quarters]
    print(f'TRAIN: {train_data.index}, TEST: {valid_data.index}')
    splits_test['train'].append(train_data)
    splits_test['test'].append(valid_data)

    if i + 7 >= len(arima_df) - 1:
        break

TRAIN: DatetimeIndex(['2008-03-31', '2008-06-30', '2008-09-30', '2008-12-31',
               '2009-03-31', '2009-06-30', '2009-09-30', '2009-12-31',
               '2010-03-31', '2010-06-30', '2010-09-30', '2010-12-31',
               '2011-03-31', '2011-06-30', '2011-09-30', '2011-12-31',
               '2012-03-31', '2012-06-30', '2012-09-30', '2012-12-31',
               '2013-03-31', '2013-06-30', '2013-09-30', '2013-12-31',
               '2014-03-31', '2014-06-30', '2014-09-30', '2014-12-31',
               '2015-03-31', '2015-06-30', '2015-09-30', '2015-12-31',
               '2016-03-31', '2016-06-30', '2016-09-30', '2016-12-31',
               '2017-03-31', '2017-06-30', '2017-09-30', '2017-12-31',
               '2018-03-31', '2018-06-30', '2018-09-30', '2018-12-31',
               '2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31',
               '2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31',
               '2021-03-31'],
              dtype='datetime64[ns]', na

## Best ACF from cross validation

### ACF

In [ ]:
# VALIDATION SPLITS
# Initialize variables to store the best ACF and corresponding lag order
best_acf = -np.inf
best_lag = None

# Iterate through different lag orders
for lag in range(1, 21):
    f = 1
    acf_values = []  # To store ACF values for each fold

    # Iterate through train-validation splits
    for train_data, valid_data in zip(splits_val['train'], splits_val['val']):
        # Fit AutoRegressive model
        model = sm.tsa.AutoReg(train_data, lags=lag).fit()

        # Calculate autocorrelation function
        acf = sm.tsa.stattools.acf(model.resid, fft=False, nlags=lag)[-1]
        acf_values.append(acf)

    # Calculate mean ACF across folds
    mean_acf = np.mean(acf_values)

    # Update best ACF and corresponding lag order if current ACF is better
    if mean_acf > best_acf:
        best_acf = mean_acf
        best_lag = lag

    print(f"Lag: {lag}, Mean ACF: {mean_acf}")

# Print the best lag order and its ACF
print("Best Lag:", best_lag)
print("Best ACF:", best_acf)



Lag: 1, Mean ACF: 0.13544549960558513
Lag: 2, Mean ACF: -0.2689202532108032
Lag: 3, Mean ACF: -0.15348794713634598
Lag: 4, Mean ACF: 0.1956140951081159
Lag: 5, Mean ACF: -0.051644314166709185
Lag: 6, Mean ACF: 0.07341729120233542
Lag: 7, Mean ACF: -0.08433294980353566
Lag: 8, Mean ACF: -0.07504712654537189
Lag: 9, Mean ACF: -0.13463258813815732
Lag: 10, Mean ACF: -0.09681331927793188
Lag: 11, Mean ACF: -0.0007350631570215522
Lag: 12, Mean ACF: -0.07693178556995015
Lag: 13, Mean ACF: 0.036757857203722655
Lag: 14, Mean ACF: 0.11910180029572345
Lag: 15, Mean ACF: -0.1221871186799547
Lag: 16, Mean ACF: -0.09063396054217875
Lag: 17, Mean ACF: -0.0880220510734408
Lag: 18, Mean ACF: -0.07136416863787719
Lag: 19, Mean ACF: -0.09321849699350042
Lag: 20, Mean ACF: -0.04055955170352693
Best Lag: 4
Best ACF: 0.1956140951081159


In [ ]:
# TEST SPLITS
# Initialize variables to store the best ACF and corresponding lag order
best_acf = -np.inf
best_lag_test = None

# Iterate through different lag orders
for lag in range(1, 21):
    f = 1
    acf_values = []  # To store ACF values for each fold

    # Iterate through train-validation splits
    for train_data, valid_data in zip(splits_test['train'], splits_test['test']):
        # Fit AutoRegressive model
        model = sm.tsa.AutoReg(train_data, lags=lag).fit()

        # Calculate autocorrelation function
        acf = sm.tsa.stattools.acf(model.resid, fft=False, nlags=lag)[-1]
        acf_values.append(acf)

    # Calculate mean ACF across folds
    mean_acf = np.mean(acf_values)

    # Update best ACF and corresponding lag order if current ACF is better
    if mean_acf > best_acf:
        best_acf = mean_acf
        best_lag_test = lag

    print(f"Lag: {lag}, Mean ACF: {mean_acf}")

# Print the best lag order and its ACF
print("Best Lag:", best_lag_test)
print("Best ACF:", best_acf)


Lag: 1, Mean ACF: 0.12944422721909674
Lag: 2, Mean ACF: -0.34988290161539937
Lag: 3, Mean ACF: -0.18674413706591161
Lag: 4, Mean ACF: 0.1554516059110823
Lag: 5, Mean ACF: -0.08472488622121162
Lag: 6, Mean ACF: 0.07532536639044082
Lag: 7, Mean ACF: -0.08547480316660216
Lag: 8, Mean ACF: -0.08577211959383005
Lag: 9, Mean ACF: -0.06711419414270947
Lag: 10, Mean ACF: -0.12297372423590139
Lag: 11, Mean ACF: -0.034460989464824024
Lag: 12, Mean ACF: -0.07305510917176403
Lag: 13, Mean ACF: -0.031280307255256545
Lag: 14, Mean ACF: 0.06670851522674257
Lag: 15, Mean ACF: -0.19970024424776767
Lag: 16, Mean ACF: -0.1185424959836251
Lag: 17, Mean ACF: -0.09077222913263172
Lag: 18, Mean ACF: 0.011570741936345687
Lag: 19, Mean ACF: -0.08468250194731113
Lag: 20, Mean ACF: -0.03173801303725931
Best Lag: 4
Best ACF: 0.1554516059110823


## Regression data

In [ ]:
# Load CSV data into a DataFrame
data = pd.read_csv('Economic_indicators_final_csv.csv')

# Drop consolidated sales
data = data.drop(columns=['NCN_sales_cons'])

pd.set_option('display.max_columns', None)
data.head(10)

,quarter,NCN_sales_est,Order_book,GDP_households,GDP_government,GDP_fixed_capital,GDP_export,GDP_import,Unemployment_rate,Employment_rate,Gross_monthly_wage,CPI,PPI,Current_account_balance,Current_account_balance_GDP,FDI_balance,General_gov_net,General_gov_budget_balance,General_gov_invest,Conf_industrial,Conf_retail,Conf_service,Conf_consumer,Econ_sent_ind,Ind_BSI,Serv_BSI,Retail_volume_idx,Industrial_volume_idx,New_car_reg,Apt_sales,Building_permit_dwellings,Building_permit_nonres,Productivity_per_employee,Productivity_per_hour_worked,Enterprises_total_profit,Enterprises_invest_build,Loans_granted_to_nonfin_co,M1
0,2003-Q4,NaN,46494.0,1917.0,886.0,1089.0,1816.0,1964.0,9.1,57.7,455.0,0.16,0.49,-351.3,-15.3,-87.6,-76.9,-3.3,170.6,12.0,16.2,26.2,-3.0,108.8,47.1,31.3,67.5,60.4,3894.0,5393.0,131.8,194.9,4.3,1.1,406920.0,192888.0,1490.4,2269.0
1,2004-Q1,NaN,52520.0,1827.0,679.0,857.0,1393.0,1608.0,10.3,56.5,431.0,0.70,0.96,-348.1,-15.1,-127.7,128.5,5.6,50.7,10.3,20.3,23.9,-6.1,107.1,51.2,30.8,58.2,57.5,3920.0,4389.0,139.2,145.0,3.1,4.8,355768.0,153388.0,1620.2,2269.0
2,2004-Q2,NaN,52506.0,1944.0,840.0,1065.0,2030.0,2264.0,10.2,56.9,474.0,2.47,1.28,-389.8,-16.3,-117.8,-6.5,-0.3,87.7,14.2,26.5,25.9,-7.8,109.1,43.3,29.8,68.9,64.3,5062.0,4980.0,503.0,378.7,3.6,-0.4,443906.0,172245.0,1737.3,2482.8
3,2004-Q3,NaN,55196.0,1945.0,661.0,1124.0,2283.0,2139.0,10.5,56.6,449.0,0.62,1.26,-53.7,-2.2,-53.5,149.3,6.0,109.3,12.1,20.2,17.2,-5.0,107.1,43.7,36.7,71.7,64.2,3735.0,4894.0,153.4,217.6,4.4,11.2,460135.0,178632.0,1879.1,2542.2
4,2004-Q4,NaN,51902.0,2136.0,913.0,1129.0,2289.0,2341.0,9.5,56.7,492.0,0.74,0.23,-374.1,-14.5,-255.6,-37.2,-1.4,181.5,8.7,19.0,9.3,-2.2,103.9,40.7,36.5,75.5,67.2,3797.0,5552.0,148.0,116.3,3.2,-2.4,383585.0,179336.0,2085.9,2700.9
5,2005-Q1,18702.0,64403.0,1961.0,699.0,942.0,1987.0,1999.0,9.6,56.7,475.0,0.69,0.52,-208.0,-8.2,-854.4,-16.2,-0.6,59.4,12.7,19.6,16.9,-1.4,107.9,42.3,35.9,67.0,63.5,4363.0,5117.0,88.8,213.6,-1.5,-0.3,402813.0,156287.0,2328.8,2750.6
6,2005-Q2,22122.0,76905.0,2120.0,811.0,1175.0,2388.0,2393.0,8.4,58.2,530.0,1.43,-0.18,-209.6,-7.4,-845.7,126.4,4.4,108.1,12.5,21.4,27.5,-0.7,110.4,44.3,34.8,78.1,73.3,5843.0,6793.0,315.5,317.2,9.4,7.8,580104.0,197917.0,2567.0,3239.0
7,2005-Q3,34660.0,102218.0,2154.0,702.0,1349.0,2528.0,2538.0,7.0,58.5,498.0,1.38,0.72,-245.3,-8.4,-175.8,137.2,4.7,135.5,14.5,24.7,23.2,1.9,111.4,34.5,28.5,82.7,70.9,4828.0,8016.0,251.3,206.1,1.7,5.4,611521.0,255296.0,2967.7,3301.8
8,2005-Q4,24577.0,78761.0,2390.0,972.0,1354.0,2683.0,2816.0,7.1,59.1,555.0,0.44,0.96,-321.6,-10.5,157.9,-119.8,-3.9,222.7,17.9,25.7,27.9,6.7,115.3,35.3,25.4,86.5,75.4,4609.0,8605.0,263.1,239.6,3.7,2.0,598326.0,306242.0,3379.7,3682.7
9,2006-Q1,20840.0,113503.0,2234.0,752.0,1216.0,2296.0,2555.0,6.6,60.6,549.0,1.08,1.63,-482.3,-16.0,-182.0,28.1,0.9,84.9,14.6,27.5,28.0,9.6,114.5,34.6,23.2,78.1,70.4,5230.0,6970.0,217.7,201.8,-2.3,-1.4,568766.0,229217.0,3720.8,3821.2


In [ ]:
data.tail(2)

,quarter,NCN_sales_est,Order_book,GDP_households,GDP_government,GDP_fixed_capital,GDP_export,GDP_import,Unemployment_rate,Employment_rate,Gross_monthly_wage,CPI,PPI,Current_account_balance,Current_account_balance_GDP,FDI_balance,General_gov_net,General_gov_budget_balance,General_gov_invest,Conf_industrial,Conf_retail,Conf_service,Conf_consumer,Econ_sent_ind,Ind_BSI,Serv_BSI,Retail_volume_idx,Industrial_volume_idx,New_car_reg,Apt_sales,Building_permit_dwellings,Building_permit_nonres,Productivity_per_employee,Productivity_per_hour_worked,Enterprises_total_profit,Enterprises_invest_build,Loans_granted_to_nonfin_co,M1
79,2023-Q3,75548.0,251155.0,3227.0,1114.0,2144.0,4948.0,5471.0,7.3,69.1,1812.0,0.19,-1.06,-523.3,-5.6,90.8,-202.7,-2.2,626.0,-21.3,-8.0,-12.3,-30.0,80.5,74.6,58.0,123.0,111.7,5361.0,5424.0,151.2,231.2,0.2,-1.3,1069942.0,501179.0,9858.6,18014.0
80,2023-Q4,73766.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data[['Year', 'Quarter']] = data['quarter'].str.split('-', expand=True)
data['Year'] = data['Year'].astype(int)
data['Quarter'] = data['Quarter'].str.extract('(\d+)').astype(int)
data = data.drop('quarter', axis=1)
# Rearrange columns
data = data[['Year', 'Quarter'] + [col for col in data.columns if col not in ['Year', 'Quarter']]]
data

,Year,Quarter,NCN_sales_est,Order_book,GDP_households,GDP_government,GDP_fixed_capital,GDP_export,GDP_import,Unemployment_rate,Employment_rate,Gross_monthly_wage,CPI,PPI,Current_account_balance,Current_account_balance_GDP,FDI_balance,General_gov_net,General_gov_budget_balance,General_gov_invest,Conf_industrial,Conf_retail,Conf_service,Conf_consumer,Econ_sent_ind,Ind_BSI,Serv_BSI,Retail_volume_idx,Industrial_volume_idx,New_car_reg,Apt_sales,Building_permit_dwellings,Building_permit_nonres,Productivity_per_employee,Productivity_per_hour_worked,Enterprises_total_profit,Enterprises_invest_build,Loans_granted_to_nonfin_co,M1
0,2003,4,NaN,46494.0,1917.0,886.0,1089.0,1816.0,1964.0,9.1,57.7,455.0,0.16,0.49,-351.3,-15.3,-87.6,-76.9,-3.3,170.6,12.0,16.2,26.2,-3.0,108.8,47.1,31.3,67.5,60.4,3894.0,5393.0,131.8,194.9,4.3,1.1,406920.0,192888.0,1490.4,2269.0
1,2004,1,NaN,52520.0,1827.0,679.0,857.0,1393.0,1608.0,10.3,56.5,431.0,0.70,0.96,-348.1,-15.1,-127.7,128.5,5.6,50.7,10.3,20.3,23.9,-6.1,107.1,51.2,30.8,58.2,57.5,3920.0,4389.0,139.2,145.0,3.1,4.8,355768.0,153388.0,1620.2,2269.0
2,2004,2,NaN,52506.0,1944.0,840.0,1065.0,2030.0,2264.0,10.2,56.9,474.0,2.47,1.28,-389.8,-16.3,-117.8,-6.5,-0.3,87.7,14.2,26.5,25.9,-7.8,109.1,43.3,29.8,68.9,64.3,5062.0,4980.0,503.0,378.7,3.6,-0.4,443906.0,172245.0,1737.3,2482.8
3,2004,3,NaN,55196.0,1945.0,661.0,1124.0,2283.0,2139.0,10.5,56.6,449.0,0.62,1.26,-53.7,-2.2,-53.5,149.3,6.0,109.3,12.1,20.2,17.2,-5.0,107.1,43.7,36.7,71.7,64.2,3735.0,4894.0,153.4,217.6,4.4,11.2,460135.0,178632.0,1879.1,2542.2
4,2004,4,NaN,51902.0,2136.0,913.0,1129.0,2289.0,2341.0,9.5,56.7,492.0,0.74,0.23,-374.1,-14.5,-255.6,-37.2,-1.4,181.5,8.7,19.0,9.3,-2.2,103.9,40.7,36.5,75.5,67.2,3797.0,5552.0,148.0,116.3,3.2,-2.4,383585.0,179336.0,2085.9,2700.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,2022,4,79376.0,149799.0,3373.0,1373.0,2362.0,5666.0,6008.0,5.4,69.6,1735.0,0.39,-0.06,-298.8,-3.1,-615.1,-292.1,-3.0,608.8,-11.3,-3.9,-11.4,-35.4,83.5,57.6,40.7,134.4,123.8,4276.0,5443.0,168.6,293.0,3.0,4.0,1107262.0,463394.0,9373.4,21520.0
77,2023,1,46742.0,199947.0,3110.0,1094.0,1594.0,5185.0,5448.0,5.3,69.4,1741.0,1.57,-0.94,-76.9,-0.9,-615.1,-382.7,-4.3,361.0,-13.6,-1.9,-11.8,-28.7,83.7,59.6,48.1,112.6,119.7,5726.0,4717.0,127.4,250.1,-11.3,-9.4,977586.0,415832.0,9303.6,20532.0
78,2023,2,76021.0,256328.0,3248.0,1147.0,1707.0,5445.0,5690.0,6.7,69.2,1872.0,2.84,-1.36,-283.9,-3.0,-219.4,35.9,0.4,487.8,-18.1,-2.4,-12.5,-28.8,81.9,68.2,49.0,122.4,119.2,6511.0,5894.0,109.4,245.6,8.1,8.9,1280858.0,379544.0,9445.5,19337.0
79,2023,3,75548.0,251155.0,3227.0,1114.0,2144.0,4948.0,5471.0,7.3,69.1,1812.0,0.19,-1.06,-523.3,-5.6,90.8,-202.7,-2.2,626.0,-21.3,-8.0,-12.3,-30.0,80.5,74.6,58.0,123.0,111.7,5361.0,5424.0,151.2,231.2,0.2,-1.3,1069942.0,501179.0,9858.6,18014.0


### Extend the dataframe to accommodate lags

In [ ]:
# Determine the last year and quarter in the DataFrame
last_year = data['Year'].iloc[-1]
last_quarter = data['Quarter'].iloc[-1]

# Calculate the next quarter
if last_quarter == 4:
    next_year = last_year + 1
    next_quarter = 1
else:
    next_year = last_year
    next_quarter = last_quarter + 1

# Extend the DataFrame for the next four quarters
for i in range(4):
    new_row = {'Year': next_year, 'Quarter': next_quarter}
    data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
    if next_quarter == 4:
        next_year += 1
        next_quarter = 1
    else:
        next_quarter += 1

In [ ]:
data.tail(6)

,Year,Quarter,NCN_sales_est,Order_book,GDP_households,GDP_government,GDP_fixed_capital,GDP_export,GDP_import,Unemployment_rate,Employment_rate,Gross_monthly_wage,CPI,PPI,Current_account_balance,Current_account_balance_GDP,FDI_balance,General_gov_net,General_gov_budget_balance,General_gov_invest,Conf_industrial,Conf_retail,Conf_service,Conf_consumer,Econ_sent_ind,Ind_BSI,Serv_BSI,Retail_volume_idx,Industrial_volume_idx,New_car_reg,Apt_sales,Building_permit_dwellings,Building_permit_nonres,Productivity_per_employee,Productivity_per_hour_worked,Enterprises_total_profit,Enterprises_invest_build,Loans_granted_to_nonfin_co,M1
79,2023,3,75548.0,251155.0,3227.0,1114.0,2144.0,4948.0,5471.0,7.3,69.1,1812.0,0.19,-1.06,-523.3,-5.6,90.8,-202.7,-2.2,626.0,-21.3,-8.0,-12.3,-30.0,80.5,74.6,58.0,123.0,111.7,5361.0,5424.0,151.2,231.2,0.2,-1.3,1069942.0,501179.0,9858.6,18014.0
80,2023,4,73766.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,2024,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,2024,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,2024,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,2024,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.dtypes

Year                              int64
Quarter                           int64
NCN_sales_est                   float64
Order_book                      float64
GDP_households                  float64
GDP_government                  float64
GDP_fixed_capital               float64
GDP_export                      float64
GDP_import                      float64
Unemployment_rate               float64
Employment_rate                 float64
Gross_monthly_wage              float64
CPI                             float64
PPI                             float64
Current_account_balance         float64
Current_account_balance_GDP     float64
FDI_balance                     float64
General_gov_net                 float64
General_gov_budget_balance      float64
General_gov_invest              float64
Conf_industrial                 float64
Conf_retail                     float64
Conf_service                    float64
Conf_consumer                   float64
Econ_sent_ind                   float64


In [ ]:
data.isnull().sum()

Year                            0
Quarter                         0
NCN_sales_est                   9
Order_book                      5
GDP_households                  5
GDP_government                  5
GDP_fixed_capital               5
GDP_export                      5
GDP_import                      5
Unemployment_rate               5
Employment_rate                 5
Gross_monthly_wage              5
CPI                             5
PPI                             5
Current_account_balance         5
Current_account_balance_GDP     5
FDI_balance                     5
General_gov_net                 5
General_gov_budget_balance      5
General_gov_invest              5
Conf_industrial                 5
Conf_retail                     5
Conf_service                    5
Conf_consumer                   5
Econ_sent_ind                   5
Ind_BSI                         5
Serv_BSI                        5
Retail_volume_idx               5
Industrial_volume_idx           5
New_car_reg   

### Store attributes to be lagged

In [ ]:
data.columns

Index(['Year', 'Quarter', 'NCN_sales_est', 'Order_book', 'GDP_households',
       'GDP_government', 'GDP_fixed_capital', 'GDP_export', 'GDP_import',
       'Unemployment_rate', 'Employment_rate', 'Gross_monthly_wage', 'CPI',
       'PPI', 'Current_account_balance', 'Current_account_balance_GDP',
       'FDI_balance', 'General_gov_net', 'General_gov_budget_balance',
       'General_gov_invest', 'Conf_industrial', 'Conf_retail', 'Conf_service',
       'Conf_consumer', 'Econ_sent_ind', 'Ind_BSI', 'Serv_BSI',
       'Retail_volume_idx', 'Industrial_volume_idx', 'New_car_reg',
       'Apt_sales', 'Building_permit_dwellings', 'Building_permit_nonres',
       'Productivity_per_employee', 'Productivity_per_hour_worked',
       'Enterprises_total_profit', 'Enterprises_invest_build',
       'Loans_granted_to_nonfin_co', 'M1'],
      dtype='object')

In [ ]:
# Starting from 4th, ie exclude Year, Quarter and NCN_sales_est
attributes_for_lag = data.columns.tolist()[3:]
attributes_for_lag

['Order_book',
 'GDP_households',
 'GDP_government',
 'GDP_fixed_capital',
 'GDP_export',
 'GDP_import',
 'Unemployment_rate',
 'Employment_rate',
 'Gross_monthly_wage',
 'CPI',
 'PPI',
 'Current_account_balance',
 'Current_account_balance_GDP',
 'FDI_balance',
 'General_gov_net',
 'General_gov_budget_balance',
 'General_gov_invest',
 'Conf_industrial',
 'Conf_retail',
 'Conf_service',
 'Conf_consumer',
 'Econ_sent_ind',
 'Ind_BSI',
 'Serv_BSI',
 'Retail_volume_idx',
 'Industrial_volume_idx',
 'New_car_reg',
 'Apt_sales',
 'Building_permit_dwellings',
 'Building_permit_nonres',
 'Productivity_per_employee',
 'Productivity_per_hour_worked',
 'Enterprises_total_profit',
 'Enterprises_invest_build',
 'Loans_granted_to_nonfin_co',
 'M1']

### Create lagged values of -1 to -8 quarters

In [ ]:
# Exclude 'NCN_sales_est'
attributes = data.columns[3:]

# Number of lagged values to create
num_lags = 8

# Create lagged columns for each attribute
for attribute in attributes:
    for lag in range(1, num_lags + 1):
        new_col_name = f"{attribute}_lag_{lag}"
        data[new_col_name] = data[attribute].shift(lag)

data.tail(6)

,Year,Quarter,NCN_sales_est,Order_book,GDP_households,GDP_government,GDP_fixed_capital,GDP_export,GDP_import,Unemployment_rate,Employment_rate,Gross_monthly_wage,CPI,PPI,Current_account_balance,Current_account_balance_GDP,FDI_balance,General_gov_net,General_gov_budget_balance,General_gov_invest,Conf_industrial,Conf_retail,Conf_service,Conf_consumer,Econ_sent_ind,Ind_BSI,Serv_BSI,Retail_volume_idx,Industrial_volume_idx,New_car_reg,Apt_sales,Building_permit_dwellings,Building_permit_nonres,Productivity_per_employee,Productivity_per_hour_worked,Enterprises_total_profit,Enterprises_invest_build,Loans_granted_to_nonfin_co,M1,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag

In [ ]:
data.head(13)

,Year,Quarter,NCN_sales_est,Order_book,GDP_households,GDP_government,GDP_fixed_capital,GDP_export,GDP_import,Unemployment_rate,Employment_rate,Gross_monthly_wage,CPI,PPI,Current_account_balance,Current_account_balance_GDP,FDI_balance,General_gov_net,General_gov_budget_balance,General_gov_invest,Conf_industrial,Conf_retail,Conf_service,Conf_consumer,Econ_sent_ind,Ind_BSI,Serv_BSI,Retail_volume_idx,Industrial_volume_idx,New_car_reg,Apt_sales,Building_permit_dwellings,Building_permit_nonres,Productivity_per_employee,Productivity_per_hour_worked,Enterprises_total_profit,Enterprises_invest_build,Loans_granted_to_nonfin_co,M1,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag

### Remove original attributes and rows to start with Q1 2008, create AR attribute

---



In [ ]:
# Remove original attributes from the DataFrame
data = data.drop(attributes_for_lag, axis=1)
data.head(6)

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser

In [ ]:
data.tail(6)

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser

In [ ]:
# #create lag with best lag based on ACF for NCN_sales_est

lagged_NCN_sales_est = data['NCN_sales_est'].shift(best_lag)

# Append the lagged variable to your DataFrame
data[f'NCN_sales_est_lagged_{best_lag}'] = lagged_NCN_sales_est

In [ ]:
data

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser

In [ ]:
# Remove first 17 rows so that dataset starts without NaN values and from Q1 2008
data = data.iloc[17:]
data = data.reset_index(drop=True)
data

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser

## Separate prediction dataset, create dummies for quarters

In [ ]:
data_pred = data.iloc[-5:]
data_pred = data_pred.reset_index(drop=True)

# Create dummies for quarters
quarter_dummies = pd.get_dummies(data_pred['Quarter'], prefix='Quarter')
data_pred = pd.concat([data_pred, quarter_dummies], axis=1)

data_pred.to_csv('data_pred_ar_v2.csv', index=False)
data_pred

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser

## Separate train and validation dataset, create dummies

In [ ]:
trainvalidtest = data.iloc[:-4]

#Create dummies for quarters
quarter_dummies = pd.get_dummies(trainvalidtest['Quarter'], prefix='Quarter')
trainvalidtest = pd.concat([trainvalidtest, quarter_dummies], axis=1)

trainvalidtest.to_csv('trainvalidtest_ar_v2.csv', index=False)
trainvalidtest

,Year,Quarter,NCN_sales_est,Order_book_lag_1,Order_book_lag_2,Order_book_lag_3,Order_book_lag_4,Order_book_lag_5,Order_book_lag_6,Order_book_lag_7,Order_book_lag_8,GDP_households_lag_1,GDP_households_lag_2,GDP_households_lag_3,GDP_households_lag_4,GDP_households_lag_5,GDP_households_lag_6,GDP_households_lag_7,GDP_households_lag_8,GDP_government_lag_1,GDP_government_lag_2,GDP_government_lag_3,GDP_government_lag_4,GDP_government_lag_5,GDP_government_lag_6,GDP_government_lag_7,GDP_government_lag_8,GDP_fixed_capital_lag_1,GDP_fixed_capital_lag_2,GDP_fixed_capital_lag_3,GDP_fixed_capital_lag_4,GDP_fixed_capital_lag_5,GDP_fixed_capital_lag_6,GDP_fixed_capital_lag_7,GDP_fixed_capital_lag_8,GDP_export_lag_1,GDP_export_lag_2,GDP_export_lag_3,GDP_export_lag_4,GDP_export_lag_5,GDP_export_lag_6,GDP_export_lag_7,GDP_export_lag_8,GDP_import_lag_1,GDP_import_lag_2,GDP_import_lag_3,GDP_import_lag_4,GDP_import_lag_5,GDP_import_lag_6,GDP_import_lag_7,GDP_import_lag_8,Unemployment_rate_lag_1,Unemployment_rate_lag_2,Unemployment_rate_lag_3,Unemployment_rate_lag_4,Unemployment_rate_lag_5,Unemployment_rate_lag_6,Unemployment_rate_lag_7,Unemployment_rate_lag_8,Employment_rate_lag_1,Employment_rate_lag_2,Employment_rate_lag_3,Employment_rate_lag_4,Employment_rate_lag_5,Employment_rate_lag_6,Employment_rate_lag_7,Employment_rate_lag_8,Gross_monthly_wage_lag_1,Gross_monthly_wage_lag_2,Gross_monthly_wage_lag_3,Gross_monthly_wage_lag_4,Gross_monthly_wage_lag_5,Gross_monthly_wage_lag_6,Gross_monthly_wage_lag_7,Gross_monthly_wage_lag_8,CPI_lag_1,CPI_lag_2,CPI_lag_3,CPI_lag_4,CPI_lag_5,CPI_lag_6,CPI_lag_7,CPI_lag_8,PPI_lag_1,PPI_lag_2,PPI_lag_3,PPI_lag_4,PPI_lag_5,PPI_lag_6,PPI_lag_7,PPI_lag_8,Current_account_balance_lag_1,Current_account_balance_lag_2,Current_account_balance_lag_3,Current_account_balance_lag_4,Current_account_balance_lag_5,Current_account_balance_lag_6,Current_account_balance_lag_7,Current_account_balance_lag_8,Current_account_balance_GDP_lag_1,Current_account_balance_GDP_lag_2,Current_account_balance_GDP_lag_3,Current_account_balance_GDP_lag_4,Current_account_balance_GDP_lag_5,Current_account_balance_GDP_lag_6,Current_account_balance_GDP_lag_7,Current_account_balance_GDP_lag_8,FDI_balance_lag_1,FDI_balance_lag_2,FDI_balance_lag_3,FDI_balance_lag_4,FDI_balance_lag_5,FDI_balance_lag_6,FDI_balance_lag_7,FDI_balance_lag_8,General_gov_net_lag_1,General_gov_net_lag_2,General_gov_net_lag_3,General_gov_net_lag_4,General_gov_net_lag_5,General_gov_net_lag_6,General_gov_net_lag_7,General_gov_net_lag_8,General_gov_budget_balance_lag_1,General_gov_budget_balance_lag_2,General_gov_budget_balance_lag_3,General_gov_budget_balance_lag_4,General_gov_budget_balance_lag_5,General_gov_budget_balance_lag_6,General_gov_budget_balance_lag_7,General_gov_budget_balance_lag_8,General_gov_invest_lag_1,General_gov_invest_lag_2,General_gov_invest_lag_3,General_gov_invest_lag_4,General_gov_invest_lag_5,General_gov_invest_lag_6,General_gov_invest_lag_7,General_gov_invest_lag_8,Conf_industrial_lag_1,Conf_industrial_lag_2,Conf_industrial_lag_3,Conf_industrial_lag_4,Conf_industrial_lag_5,Conf_industrial_lag_6,Conf_industrial_lag_7,Conf_industrial_lag_8,Conf_retail_lag_1,Conf_retail_lag_2,Conf_retail_lag_3,Conf_retail_lag_4,Conf_retail_lag_5,Conf_retail_lag_6,Conf_retail_lag_7,Conf_retail_lag_8,Conf_service_lag_1,Conf_service_lag_2,Conf_service_lag_3,Conf_service_lag_4,Conf_service_lag_5,Conf_service_lag_6,Conf_service_lag_7,Conf_service_lag_8,Conf_consumer_lag_1,Conf_consumer_lag_2,Conf_consumer_lag_3,Conf_consumer_lag_4,Conf_consumer_lag_5,Conf_consumer_lag_6,Conf_consumer_lag_7,Conf_consumer_lag_8,Econ_sent_ind_lag_1,Econ_sent_ind_lag_2,Econ_sent_ind_lag_3,Econ_sent_ind_lag_4,Econ_sent_ind_lag_5,Econ_sent_ind_lag_6,Econ_sent_ind_lag_7,Econ_sent_ind_lag_8,Ind_BSI_lag_1,Ind_BSI_lag_2,Ind_BSI_lag_3,Ind_BSI_lag_4,Ind_BSI_lag_5,Ind_BSI_lag_6,Ind_BSI_lag_7,Ind_BSI_lag_8,Serv_BSI_lag_1,Serv_BSI_lag_2,Serv_BSI_lag_3,Serv_BSI_lag_4,Serv_BSI_lag_5,Serv_BSI_lag_6,Serv_BSI_lag_7,Ser